In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy
import scipy.io
from scipy.linalg import inv

np.set_printoptions(threshold=10000)
plt.rcParams['font.sans-serif'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'cm'

# define a function to import data like MATLAB
def importdata(path):
    f = open(path)
    line = f.readline()
    data = []
    while line:
        num = list(map(float, line.split()))
        data.append(num)
        line = f.readline()
    f.close()
    return np.array(data)

In [3]:
# solving the relative change in prices (equation 11)
# replication of file "PH.m"
def PH(wages_N, tau_hat, T, B, G, Din, J, N, maxit, tol):
    # reformating theta vector (note that here theta vector refers to 1/θ)
    LT = np.ndarray([1240, 1])
    for j in range(J):
        LT[N*j : N*(j+1)] = np.ones([N, 1]) * T[j] # vector with 1280 elements
    
    # initializing vectors of ex-post wage and price factors
    wf0 = wages_N # the initial value of wage_hat
    pf0 = np.ones([J, N]) # the initial value of phat
    pfmax = 1
    it = 1
    # solving the wage and prices
    while (it <= maxit) & (pfmax > tol):
        lw = np.log(wf0) # log of wage
        lp = np.log(pf0) # log of intermediates' price
        
        # calculating the log cost of one unit input bundle
        lc = np.ndarray([40, 31])
        for n in range(N):
            lc[:, n] = B[:, n] * lw[n] + (1 - B[:, n]) * (G[n*J : (n+1)*J, :].T) @ lp[:, n]
        c = np.exp(lc)
        Din_om = Din * (tau_hat ** (-1 / (LT @ np.ones([1,N]))))
        # Din is the trade share, and Din_om refers to the first two terms in the brackets of (11).
        
        # calculating phat
        phat = np.ndarray([J, N]) 
        for j in range(J):
            for n in range(N):
                phat[j, n] = Din_om[n+N*j, :] @ (c[j, :] ** ((- 1 / T[j])).T)
                if phat[j, n] == 0:
                    phat[j, n] = 1 
                else:
                    phat[j, n] = phat[j, n] ** (- T[j]) 
        
        pfdev = abs(phat - pf0) # checking tolerance
        pf0 = phat
        pfmax = pfdev.max()
        it += 1 # checking iteration times

    return pf0, c

In [4]:
# solving the counterfactual bilateral trade shares (equation 12 × Din)
# replication of file "Dinprime.m"
def Dinprime(Din, tau_hat, c, T, J, N):
    # reformating theta vector 
    LT = np.ndarray([1240,1])
    for j in range(J):
        LT[N*j : N*(j+1)] = np.ones([N, 1]) * T[j]

    # calculating the change in c with power
    cp = np.ndarray([40, 31])
    for n in range(N):
        cp[:, n] = c[:, n] ** (- 1 / T[j])

    Din_om = Din * (tau_hat ** (- 1 / (LT @ np.ones([1, N]))))

    DD = np.ndarray([1240, 31])
    for n in range(N):
        DD[n::N, :] = Din_om[n::N, :] * cp
    
    phat = DD.sum(axis=1).reshape([1240, 1]) ** (- LT)

    Dinp = np.ndarray([1240, 31])
    for n in range(N):
        Dinp[:, n] = DD[:, n] * (phat ** (1 / LT)).reshape([1240,])

    return Dinp

In [5]:
# solving the total expenditure (equation 13)
# replication of file "expenditure.m"
# see Appendix C. step 4 for detail
def expenditure(alphas, B, G, Dinp, taup, Fp, VAn, wf0, Sn, J, N):
    IA = np.ndarray([J*N, J*N])
    I_F = 1 - Fp
    for n in range(N):
        IA[J*n:J*(n+1), J*n:J*(n+1)] = alphas[:, n].reshape([40, 1]) @ I_F[:, n].reshape([1, 40])

    Pit = Dinp / taup
    Bt = 1 - B
    BP = np.ndarray([1240, 31])
    for j in range(J):
        BP[N*j : N*(j+1), :] = (np.ones([N, 1]) @ Bt[j, :].reshape([1, 31])) * Pit[N*j : N*(j+1), :]

    NBP = np.ndarray([31, 1240])
    for j in range(N):
        for n in range(N):
            NBP[j, J*n : J*(n+1)] = BP[n::N, j].reshape([1,40])

    NNBP = np.kron(NBP, np.ones([J ,1]))
    GG = np.kron(np.ones([1, N]), G)
    GP = GG * NNBP

    OM = np.eye(J*N) - (GP + IA)
    Vb = alphas * np.kron(np.ones([J, 1]), (wf0 * VAn).T)
    Vb = Vb.T.reshape([J*N, 1])
    Bb = - alphas * (Sn @ np.ones([1, J])).T
    Bb = Bb.T.reshape([J*N, 1])

    DD1 = inv(OM) @ Vb
    DD2 = inv(OM) @ Bb
    PQ = DD1 + DD2
    PQ = PQ.reshape([N, J]).T

    return PQ

In [6]:
# solving the market clearing condition (equation 14)
# replication of file "LMC.m"
def LMC(Xp, Dinp, J, N, B, VAL):
    PQ_vec = Xp.reshape([J*N, 1])

    DDinpt = np.ndarray([1240, 31])
    for n in range(N):
        DDinpt[:, n] = Dinp[:, n] * PQ_vec.reshape([J*N,])

    DDDinpt = np.ndarray([40, 31]) 
    for j in range(J):
        DDDinpt[j, :] = DDinpt[N*j : N*(j+1), :].sum(axis=0)
    
    aux4 = B * DDDinpt # valued added in country n and sector j
    aux5 = aux4.sum(axis=0) # value added in country n
    aux5 = aux5.reshape([31, 1])
    wf0 = aux5 / VAL

    return wf0

In [7]:
# calculating the counterfactual equilibrium
# replication of file "equilibrium_LC.m"
def equilibrium_LC(tau_hat, taup, alphas, T, B, G, Din, J, N, maxit, tol, VAn, Sn, vfactor):
    # step 1: initializing vectors of ex-post wage and price facctors
    wf0 = np.ones([N, 1])
    pf0 = np.ones([J, N])
    wfmax = 1
    e = 1

    while (e <= maxit) & (wfmax > tol):
    # step 2: solving the price factors in each sector and each country
        pf0, c = PH(wf0, tau_hat, T, B, G, Din, J, N, maxit, tol)

    # setp 3: calculating counterfactual trade shares
        Dinp = Dinprime(Din, tau_hat, c, T, J, N)
        Dinp_om = Dinp / taup

    # step 4: solving the expenditure in each sector and each country
        Fp = np.ndarray([40, 31])
        for j in range(J):
            Fp[j, :] = (Dinp[N*j : N*(j+1)] / taup[N*j : N*(j+1), :]).sum(axis=1)
        PQ = expenditure(alphas, B, G, Dinp, taup, Fp, VAn, wf0, Sn, J, N) # expenditure matrix

    # step 5: using LMC to iterating the wage vectors
        wf1 = LMC(PQ, Dinp_om, J, N, B, VAn)

    # setp 6: verifying the market clearing conditions
        ZW = wf1 - wf0 # excess function of labor market balance

        PQ_vec = PQ.reshape([J*N, 1]) # 1240×1 vector of expenditures
        
        DP = np.ndarray([1240, 31])
        for n in range(N):
            DP[:, n] = Dinp_om[:, n] * PQ_vec.reshape([J*N,])
        LHS = DP.sum(axis=0).reshape([31, 1]) # exports

        PF = PQ * Fp
        RHS = PF.sum(axis=0).reshape([31, 1]) # imports

        Snp = (RHS - LHS + Sn) # excess function of trade balance
        ZW2 = - (RHS - LHS + Sn) / VAn

        wf1 = wf0 * (1 - vfactor * ZW2 / wf0)
        wfmax = abs(wf1 - wf0).sum()
        wfmax = abs(Snp).sum()

        wf0 = wf1
        print("iteration times: " + str(e), end='\r')
        e += 1
        
    return wf0, pf0, PQ, Fp, Dinp, ZW, Snp, c, DP, PF, e